In [28]:
from pathlib import Path
from decode_fish.funcs.utils import *

def make_sweep_str(model_path, pfile, pars, group):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    base_str = base_str + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        base_str += ' ' + k + f'={pars[k]}'
        name += k.split('.')[-1]+f':{pars[k]}x'
    name = name[:-1]
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(model_path, pfile, name, group):
    
    base_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    base_str = base_str + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [31]:
model_path = Path('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/N2_352/')
group = 'sweep_fb6'
pfile = 'sim_density_fac1_b6'

variable_col = param_iter()
variable_col.add('microscope.int_mu',1.,2.,3.,4.)

par_prod = variable_col.param_product()

# name = 'Ref_SI'

In [32]:
Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(model_path, pfile, pars, group)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_density_fac1_b6 microscope.int_mu=1.0 run_name=int_mu:1.0 output.group=sweep_fb6

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_density_fac1_b6 microscope.int_mu=2.0 run_name=int_mu:2.0 output.group=sweep_fb6

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_density_fac1_b6 microscope.int_mu=3.0 run_name=int_mu:3.0 output.group=sweep_fb6

bsub -gpu "num=1" -q gpu_any

In [72]:
# Path.mkdir(model_path/group, exist_ok=True)

# with open('run_slurm.sh', 'w') as f:
#     exec_str = make_named_str(model_path, pfile, name, group)
#     print(exec_str)
#     f.write(exec_str)